In [ ]:
# DISCLAIMER

# Les outputs plotly ne sont pas dans le notebook car ils sont très volumineux, rendant le fichier
# plus grand que la limite de taille imposée par GitHub.

# New York taxis trips

This homework is about New York taxi trips. Here is something from [Todd Schneider](https://toddwschneider.com/posts/analyzing-1-1-billion-nyc-taxi-and-uber-trips-with-a-vengeance/):

> The New York City Taxi & Limousine Commission has released a  detailed historical dataset covering over 1 billion individual taxi trips in the city from January 2009 through December 2019. 
Taken as a whole, the detailed trip-level data is more than just a vast list of taxi pickup and drop off coordinates: it's a story of a City. 
How bad is the rush hour traffic from Midtown to JFK? 
Where does the Bridge and Tunnel crowd hang out on Saturday nights?
What time do investment bankers get to work? How has Uber changed the landscape for taxis?
The dataset addresses all of these questions and many more.

The NY taxi trips dataset has been plowed by series of distinguished data scientists.
The dataset is available from on Amazon S3 (Amazon's cloud storage service).
The link for each file has the following form:

    https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_{year}-{month}.csv

There is one CSV file for each NY taxi service (`yellow`, `green`, `fhv`) and each calendar month (replacing `{year}` and `{month}` by the desired ones).
Each file is moderately large, a few gigabytes. 
The full dataset is relatively large if it has to be handled on a laptop (several hundred gigabytes).

You will focus on the `yellow` taxi service and a pair of months, from year 2015 and from year 2018. 
Between those two years, for hire vehicles services have taken off and carved a huge marketshare.

Whatever the framework you use, `CSV` files prove hard to handle. 
After downloading the appropriate files (this takes time, but this is routine), a first step will consist in converting the csv files into a more Spark friendly format such as `parquet`.

Saving into one of those formats require decisions about bucketing, partitioning and so on. Such decisions influence performance. It is your call.
Many people have been working on this dataset, to cite but a few:


- [1 billion trips with a vengeance](https://toddwschneider.com/posts/analyzing-1-1-billion-nyc-taxi-and-uber-trips-with-a-vengeance/)
- [1 billion trips with R and SQL ](http://freerangestats.info/blog/2019/12/22/nyc-taxis-sql)
- [1 billion trips with redshift](https://tech.marksblogg.com/billion-nyc-taxi-rides-redshift.html)
- [nyc-taxi](https://github.com/fmaletski/nyc-taxi-map)

Depending on your internet connection, **download the files** corresponding to **"yellow" taxis** for the years 2015 and 2018. Download **at least one month** (the same) for 2015 and 2018, if you can download all of them.

**Hint.** The 12 csv for 2015 are about 23GB in total, but the corresponding parquet file, if you can create it for all 12 months, is only about 3GB.

You **might** need the following stuff in order to work with GPS coordinates and to plot things easily.

In [ ]:
!pip install geojson geopandas plotly geopy

For this homework **we will let you decide on the tools to use** (expected for Spark) and to **find out information all by yourself** (but don't hesitate to ask questions on the `slack` channel).

## Loading data as parquet files

We want to organize the data on a per year and per service basis. 
We want to end up with one `parquet` file for each year and each taxi service, since parquet is much better than CSV files.

**Hint.** Depending on your internet connection and your laptop, you can use only the "yellow" service and use one month of 2015 and 2018

CSV files can contain corrupted lines. You may have to work in order to perform ETL (Extract-Transform-Load) in order obtain a properly typed data frame.

You are invited to proceed as follows:

1. Try to read the CSV file without imposing a schema. 
1. Inspect the inferred schema. Do you agree with Spark's typing decision?
1. Eventually correct the schema and read again the data
1. Save the data into parquet files
1. In the rest of your work, **you will only use the parquet files you created**, not the csv files (don't forget to choose a partitioning column and a number of partitions when creating the parquet files).

**Hint.** Don't forget to ask `Spark` to use all the memory and ressources from your computer.

**Hint.** Don't foreget that you should specify a partitioning column and a number of partitions when creating the parquet files.

**Hint.** Note that the schemas of the 2015 and 2018 data are different...

**Hint.** When working on this, ask you and answer to the following questions:

1. What is the `StorageLevel` of the dataframe after reading the csv files?
1. What is the number of partitions of the dataframe? 
1. Is it possible to tune this number at loading time? 
1. Why would we want to modify the number of partitions when creating the parquet files?

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import col
import pyspark.sql.functions as fn
from pyspark.sql.catalog import Catalog
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType, DoubleType, TimestampType, LongType

import plotly.express as px
import matplotlib.pyplot as plt
import json

In [ ]:
conf = SparkConf().setAppName("Spark SQL")
sc = SparkContext(conf=conf)

spark = (SparkSession
    .builder
    .appName("Spark SQL")
    .getOrCreate()
)

In [ ]:
# Lecture du fichier pour le mois d'octobre 2015
df_2015 = spark.read.csv('yellow_tripdata_2015-10.csv', header=True)

# Modification des types du schéma
df_2015 = df_2015.withColumn("VendorID", df_2015["VendorID"].cast(IntegerType()))
df_2015 = df_2015.withColumn("tpep_pickup_datetime", df_2015["tpep_pickup_datetime"].cast(TimestampType()))
df_2015 = df_2015.withColumn("tpep_dropoff_datetime", df_2015["tpep_dropoff_datetime"].cast(TimestampType()))
df_2015 = df_2015.withColumn("passenger_count", df_2015["passenger_count"].cast(IntegerType()))
df_2015 = df_2015.withColumn("trip_distance", df_2015["trip_distance"].cast(DoubleType()))
df_2015 = df_2015.withColumn("pickup_longitude", df_2015["pickup_longitude"].cast(DoubleType()))
df_2015 = df_2015.withColumn("pickup_latitude", df_2015["pickup_latitude"].cast(DoubleType()))
df_2015 = df_2015.withColumn("RatecodeID", df_2015["RatecodeID"].cast(IntegerType()))
df_2015 = df_2015.withColumn("store_and_fwd_flag", fn.when(df_2015["store_and_fwd_flag"] == "Y", True).otherwise(fn.when(df_2015["store_and_fwd_flag"] == "N", False).otherwise(None)))
df_2015 = df_2015.withColumn("dropoff_longitude", df_2015["dropoff_longitude"].cast(DoubleType()))
df_2015 = df_2015.withColumn("dropoff_latitude", df_2015["dropoff_latitude"].cast(DoubleType()))
df_2015 = df_2015.withColumn("payment_type", df_2015["payment_type"].cast(IntegerType()))
df_2015 = df_2015.withColumn("fare_amount", df_2015["fare_amount"].cast(DoubleType()))
df_2015 = df_2015.withColumn("extra", df_2015["extra"].cast(DoubleType()))
df_2015 = df_2015.withColumn("mta_tax", df_2015["mta_tax"].cast(DoubleType()))
df_2015 = df_2015.withColumn("tip_amount", df_2015["tip_amount"].cast(DoubleType()))
df_2015 = df_2015.withColumn("tolls_amount", df_2015["tolls_amount"].cast(DoubleType()))
df_2015 = df_2015.withColumn("improvement_surcharge", df_2015["improvement_surcharge"].cast(DoubleType()))
df_2015 = df_2015.withColumn("total_amount", df_2015["total_amount"].cast(DoubleType()))

# Écriture du dataframe dans un fichier parquet
df_2015.repartition(1)\
        .write\
        .mode('overwrite')\
        .format('parquet')\
        .option('path', "yellow_tripdata_2015.parquet")\
        .saveAsTable("yellow_tripdata_2015")

df_2015.printSchema()

In [ ]:
# Lecture du fichier pour le mois d'octobre 2018
df_2018 = spark.read.csv('yellow_tripdata_2018-10.csv', header=True)

# Modification des types du schéma
df_2018 = df_2018.withColumn("VendorID", df_2018["VendorID"].cast(IntegerType()))
df_2018 = df_2018.withColumn("tpep_pickup_datetime", df_2018["tpep_pickup_datetime"].cast(TimestampType()))
df_2018 = df_2018.withColumn("tpep_dropoff_datetime", df_2018["tpep_dropoff_datetime"].cast(TimestampType()))
df_2018 = df_2018.withColumn("passenger_count", df_2018["passenger_count"].cast(IntegerType()))
df_2018 = df_2018.withColumn("trip_distance", df_2018["trip_distance"].cast(DoubleType()))
df_2018 = df_2018.withColumn("RatecodeID", df_2018["RatecodeID"].cast(IntegerType()))
df_2018 = df_2018.withColumn("store_and_fwd_flag", fn.when(df_2018["store_and_fwd_flag"] == "Y", True).otherwise(fn.when(df_2018["store_and_fwd_flag"] == "N", False).otherwise(None)))
df_2018 = df_2018.withColumn("PULocationID", df_2018["PULocationID"].cast(IntegerType()))
df_2018 = df_2018.withColumn("DOLocationID", df_2018["DOLocationID"].cast(IntegerType()))
df_2018 = df_2018.withColumn("payment_type", df_2018["payment_type"].cast(IntegerType()))
df_2018 = df_2018.withColumn("fare_amount", df_2018["fare_amount"].cast(DoubleType()))
df_2018 = df_2018.withColumn("extra", df_2018["extra"].cast(DoubleType()))
df_2018 = df_2018.withColumn("mta_tax", df_2018["mta_tax"].cast(DoubleType()))
df_2018 = df_2018.withColumn("tip_amount", df_2018["tip_amount"].cast(DoubleType()))
df_2018 = df_2018.withColumn("tolls_amount", df_2018["tolls_amount"].cast(DoubleType()))
df_2018 = df_2018.withColumn("improvement_surcharge", df_2018["improvement_surcharge"].cast(DoubleType()))
df_2018 = df_2018.withColumn("total_amount", df_2018["total_amount"].cast(DoubleType()))

# Écriture du dataframe dans un fichier parquet
df_2018.repartition(1)\
        .write\
        .mode('overwrite')\
        .format('parquet')\
        .option('path', "yellow_tripdata_2018.parquet")\
        .saveAsTable("yellow_tripdata_2018")

df_2018.printSchema()

In [ ]:
# Lecture à partir d'un fichier .parquet
df_2015 = spark.read.parquet("yellow_tripdata_2015.parquet")
print('Schema 2015')

df_2018 = spark.read.parquet("yellow_tripdata_2018.parquet")
print('Schema 2018')

# Investigate (at least) one month of data in 2015

From now on, you will be using **the parquet files you created for 2015**.

We shall visualize several features of taxi traffic during one calendar month
in 2015 and the same calendar month in 2018.

**Hint.** In order to build appealing graphics, you may stick to `matplotlib + seaborn`, you can use also
`plotly`, which is used a lot to build interactive graphics, but you can use whatever you want.

The following longitudes and lattitudes encompass Newark and JFK airports, Northern Manhattan and Verazzano bridge.

In [ ]:
long_min = -74.10
long_max = -73.70
lat_min = 40.58
lat_max = 40.90

1. Using these boundaries, **filter the 2015 data** (using pickup and dropoff longitude and latitude) and count the number of trips for each value of `passenger_count` and make a plot of that.

In [ ]:
df = df_2015.where(df_2015.pickup_longitude > long_min).where(df_2015.pickup_longitude < long_max)\
            .where(df_2015.pickup_latitude > lat_min).where(df_2015.pickup_latitude < lat_max)\
            .select('passenger_count')

fig = px.histogram(df.sample(0.1).toPandas(), nbins=10)
fig.show()

Trips with $0$ or larger than $7$ passengers are pretty rare.
We suspect these to be outliers. 
We need to explore these trips further in order order to understand what might be wrong
with them

1. What's special with trips with zero passengers?
1. What's special with trips with more than $6$ passengers?
1. What is the largest distance travelled during this month? Is it the first taxi on the moon?
1. Plot the distribution of the `trip_distance` (using an histogram for instance) during year 2015. Focus on trips with non-zero trip distance and trip distance less than 30 miles.

In [ ]:
# 1.
df1 = df_2015.where(df_2015.passenger_count == 0)\
            .where(df_2015.pickup_longitude > long_min).where(df_2015.pickup_longitude < long_max)\
            .where(df_2015.pickup_latitude > lat_min).where(df_2015.pickup_latitude < lat_max)

df1.select('trip_distance', 'total_amount').describe().show()

In [ ]:
# 2.
df2 = df_2015.where(df_2015.passenger_count > 6)\
            .where(df_2015.pickup_longitude > long_min).where(df_2015.pickup_longitude < long_max)\
            .where(df_2015.pickup_latitude > lat_min).where(df_2015.pickup_latitude < lat_max)

df2.select('trip_distance', 'total_amount').describe().show()

In [ ]:
# 3.
df3 = df_2015.select('trip_distance').describe()
df3.show()

df31 = df_2015.where(df_2015.trip_distance > 3008331).select('pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count', 'trip_distance', 'total_amount')
df31.show()

In [ ]:
# 4.
df4 = df_2015.select(df_2015.trip_distance)\
            .where(df_2015.trip_distance != 0).where(df_2015.trip_distance < 30)

fig = px.histogram(df4.sample(0.1).toPandas(), nbins=100)
fig.show()

Let's look at what Spark does for these computations

1. Use the `explain` method or have a look at the [Spark UI](http://localhost:4040/SQL/) to analyze the job. You should be able to assess 
    - Parsed Logical Plan
    - Analyzed Logical Plan
    - Optimized Logical Plan
    - Physical Plan
1. Do the Analyzed Logical Plan and Optimized Logical Plan differ? Spot the differences if any. How would a RDBMS proceed with such a query?
1. How does the physical plan differ from the Optimized Logical Plan? What are the keywords you would not expects in a RDBMS? What is their meaning? 
1. Inspect the stages on [Spark UI](http://localhost:4040/stages/stage). How many *stages* are necessary to complete the Spark job? What are the roles of `HashAggregate` and `Exchange hashpartitioning`?
1. Does the physical plan perform `shuffle` operations? If yes how many?
1. What are tasks with respect to stages (in Spark language)? How many tasks are your stages made of?

In [ ]:
# 1
df1.explain(extended=True)
df2.explain(extended=True)
df3.explain(extended=True)
df31.explain(extended=True)
df4.explain(extended=True)

# 2
# For the previous query, it differs by using only one filter on the optimized plan,
# while on the analyzed and logical plan the RDBMS will use two filters : one after the other,
# then a projection as for df2 for instance.

# 3 
# Spark in the physical plan decides which is the best strategy to use after it generated the logical plan,
# the key words are PartitionFilters, PushedFilters (which are list of the filter used in that plan) and ReadSchema.

# 4
# Only one is necessary to complete the Spark job.

# 5
# No, because its a narrow transformation, using filter only, and no index as in a join.

Now, compute the following and produce relevant plots:

1. Break down the trip distance distribution for each day of week
1. Count the number of distinct pickup location
1. Compute and display tips and profits as a function of the pickup location

In [ ]:
# 1.
df = df_2015.select('tpep_pickup_datetime', 'trip_distance')
df = df.withColumn('day', fn.when(fn.dayofweek(df.tpep_pickup_datetime) == 1, "Monday")\
                          .when(fn.dayofweek(df.tpep_pickup_datetime) == 2, "Tuesday")\
                          .when(fn.dayofweek(df.tpep_pickup_datetime) == 3, "Wenesday")\
                          .when(fn.dayofweek(df.tpep_pickup_datetime) == 4, "Thursday")\
                          .when(fn.dayofweek(df.tpep_pickup_datetime) == 5, "Friday")\
                          .when(fn.dayofweek(df.tpep_pickup_datetime) == 6, "Saturday")\
                          .when(fn.dayofweek(df.tpep_pickup_datetime) == 7, "Sunday")\
                          .otherwise(None))\
                          .drop('tpep_pickup_datetime')

fig = px.histogram(df.sample(0.1).toPandas(), x="trip_distance", animation_frame="day", nbins=200)
fig.show()

In [ ]:
# 2.
df = df_2015.groupBy('pickup_longitude', 'pickup_latitude').count().count()
print(f'There is {df} distinct pickup locations.')

In [ ]:
# 3.
df = df_2015.where(df_2015.pickup_longitude > long_min).where(df_2015.pickup_longitude < long_max)\
            .where(df_2015.pickup_latitude > lat_min).where(df_2015.pickup_latitude < lat_max)\
            .where(df_2015.total_amount >= 0)\
            .where(df_2015.tip_amount >= 0)\
            .select('pickup_longitude', 'pickup_latitude', 'tip_amount', 'total_amount')

fig = px.scatter(df.sample(0.1).toPandas(), x="pickup_latitude", y="pickup_longitude", size='total_amount')
fig.show()

fig = px.scatter(df.sample(0.1).toPandas(), x="pickup_latitude", y="pickup_longitude", size='tip_amount')
fig.show()

# Investigate one month of trips data in 2015 and 2018

 Consider one month of trips data from `yellow` taxis for each year

1. Filter and cache/persist the result

## Assessing seasonalities and looking at time series

Compute and plot the following time series indexed by day of the week and hour of day:

1. The number of pickups
1. The average fare
1. The average trip duration
1. Plot the average of ongoing trips

In [ ]:
# 3.1.1 - 2015

df_days_2015 = df_2015.withColumn('hour', fn.hour(df_2015.tpep_pickup_datetime))\
                 .withColumn('day', fn.dayofweek(df_2015.tpep_pickup_datetime))\
                 .withColumn('day_week', fn.when(fn.dayofweek(df_2015.tpep_pickup_datetime) == 1, "Monday")\
                          .when(fn.dayofweek(df_2015.tpep_pickup_datetime) == 2, "Tuesday")\
                          .when(fn.dayofweek(df_2015.tpep_pickup_datetime) == 3, "Wenesday")\
                          .when(fn.dayofweek(df_2015.tpep_pickup_datetime) == 4, "Thursday")\
                          .when(fn.dayofweek(df_2015.tpep_pickup_datetime) == 5, "Friday")\
                          .when(fn.dayofweek(df_2015.tpep_pickup_datetime) == 6, "Saturday")\
                          .when(fn.dayofweek(df_2015.tpep_pickup_datetime) == 7, "Sunday")\
                          .otherwise(None))\

df = df_days_2015.groupBy('day', 'hour', 'day_week').count().orderBy('day', 'hour')

fig = px.histogram(df.toPandas(), x='hour', y='count', animation_frame='day_week', nbins=24)
fig.show()

In [ ]:
# 3.1.1 - 2018

df_days_2018 = df_2018.withColumn('hour', fn.hour(df_2018.tpep_pickup_datetime))\
                 .withColumn('day', fn.dayofweek(df_2018.tpep_pickup_datetime))\
                 .withColumn('day_week', fn.when(fn.dayofweek(df_2018.tpep_pickup_datetime) == 1, "Monday")\
                          .when(fn.dayofweek(df_2018.tpep_pickup_datetime) == 2, "Tuesday")\
                          .when(fn.dayofweek(df_2018.tpep_pickup_datetime) == 3, "Wenesday")\
                          .when(fn.dayofweek(df_2018.tpep_pickup_datetime) == 4, "Thursday")\
                          .when(fn.dayofweek(df_2018.tpep_pickup_datetime) == 5, "Friday")\
                          .when(fn.dayofweek(df_2018.tpep_pickup_datetime) == 6, "Saturday")\
                          .when(fn.dayofweek(df_2018.tpep_pickup_datetime) == 7, "Sunday")\
                          .otherwise(None))\

df = df_days_2018.groupBy('day', 'hour', 'day_week').count().orderBy('day', 'hour')

fig = px.histogram(df.toPandas(), x='hour', y='count', animation_frame='day_week', nbins=24)
fig.show()

In [ ]:
# 3.1.2 - 2015

df = df_days_2015.groupBy('day', 'hour', 'day_week').avg('fare_amount').orderBy('day', 'hour')

fig = px.line(df.toPandas(), x='hour', y='avg(fare_amount)', animation_frame='day_week')
fig.show()

In [ ]:
# 3.1.2 - 2018

df = df_days_2018.groupBy('day', 'hour', 'day_week').avg('fare_amount').orderBy('day', 'hour')

fig = px.line(df.toPandas(), x='hour', y='avg(fare_amount)', animation_frame='day_week')
fig.show()

In [ ]:
# 3.1.3 - 2015
df = df_days_2015.withColumn('time_duration', (col("tpep_dropoff_datetime").cast("long") - col('tpep_pickup_datetime').cast("long")) / 60)
df = df.where(df.time_duration > 0).groupBy('day', 'hour', 'day_week').avg('time_duration').orderBy('day', 'hour')

fig = px.line(df.toPandas(), x='hour', y='avg(time_duration)', animation_frame='day_week')
fig.show()

In [ ]:
# 3.1.3 - 2018
df = df_days_2018.withColumn('time_duration', (col("tpep_dropoff_datetime").cast("long") - col('tpep_pickup_datetime').cast("long")) / 60)
df = df.where(df.time_duration > 0).groupBy('day', 'hour', 'day_week').avg('time_duration').orderBy('day', 'hour')

fig = px.line(df.toPandas(), x='hour', y='avg(time_duration)', animation_frame='day_week')
fig.show()

## Rides to the airports

In order to find the longitude and lattitude of JFK and Newark airport as well as the longitude and magnitudes 
of Manhattan, you can use a service like [geojson.io](http://geojson.io/).
Plot the following time series, indexed the day of the week and hour of the day

1. Median duration of taxi trip leaving Midtown (Southern Manhattan) headed for JFK Airport
1. Median taxi duration of trip leaving from JFK Airport to Midtown (Southern Manhattan)

In [ ]:
# 3.2.1 - 2015

long_min_JFK = -73.82366180419922
long_max_JFK = -73.7431526184082
lat_min_JFK = 40.61799193991041
lat_max_JFK = 40.66944158747386

long_min_MT = -74.01403427124023
long_max_MT = -73.9548110961914
lat_min_MT = 40.7304783951045
lat_max_MT = 40.77430186363723

df_jfk_to_midtown = df_2015.where(df_2015.pickup_longitude > long_min_JFK).where(df_2015.pickup_longitude < long_max_JFK)\
                           .where(df_2015.pickup_latitude > lat_min_JFK).where(df_2015.pickup_latitude < lat_max_JFK)\
                           .where(df_2015.dropoff_longitude > long_min_MT).where(df_2015.dropoff_longitude < long_max_MT)\
                           .where(df_2015.dropoff_latitude > lat_min_MT).where(df_2015.dropoff_latitude < lat_max_MT)
df_jfk_to_midtown = df_jfk_to_midtown.withColumn('time_duration', (col("tpep_dropoff_datetime").cast("long") - col('tpep_pickup_datetime').cast("long")) / 60)

print('Median duration of taxi trip leaving Midtown headed for JFK Airport in 2015 : {:.2f} minutes.'.format(df_jfk_to_midtown.approxQuantile("time_duration", [0.5], 0.1)[0]))

In [ ]:
# 3.2.1 - 2018

jfk_code = 132
midtown_codes = [48, 50, 68, 90, 100, 107, 137, 161, 162, 163, 164, 170, 186, 224, 229, 230, 233, 234, 246]

df_jfk_to_midtown = df_2018.where(df_2018.DOLocationID.isin(midtown_codes))
df_jfk_to_midtown = df_jfk_to_midtown.where(df_2018.PULocationID == jfk_code)
df_jfk_to_midtown = df_jfk_to_midtown.withColumn('time_duration', (col("tpep_dropoff_datetime").cast("long") - col('tpep_pickup_datetime').cast("long")) / 60)

print('Median duration of taxi trip leaving Midtown headed for JFK Airport in 2018 : {:.2f} minutes.'.format(df_jfk_to_midtown.approxQuantile("time_duration", [0.5], 0.1)[0]))


In [ ]:
# 3.2.2 - 2015

df_midtown_to_jfk = df_2015.where(df_2015.pickup_longitude > long_min_MT).where(df_2015.pickup_longitude < long_max_MT)\
                           .where(df_2015.pickup_latitude > lat_min_MT).where(df_2015.pickup_latitude < lat_max_MT)\
                           .where(df_2015.dropoff_longitude > long_min_JFK).where(df_2015.dropoff_longitude < long_max_JFK)\
                           .where(df_2015.dropoff_latitude > lat_min_JFK).where(df_2015.dropoff_latitude < lat_max_JFK)
df_midtown_to_jfk = df_midtown_to_jfk.withColumn('time_duration', (col("tpep_dropoff_datetime").cast("long") - col('tpep_pickup_datetime').cast("long")) / 60)

print('Median taxi duration of trip leaving from JFK Airport to Midtown in 2015 : {:.2f} minutes.'.format(df_midtown_to_jfk.approxQuantile("time_duration", [0.5], 0.1)[0]))

In [ ]:
# 3.2.2 - 2018

df_midtown_to_jfk = df_2018.where(df_2018.PULocationID.isin(midtown_codes))
df_midtown_to_jfk = df_midtown_to_jfk.where(df_2018.DOLocationID == jfk_code)
df_midtown_to_jfk = df_midtown_to_jfk.withColumn('time_duration', (col("tpep_dropoff_datetime").cast("long") - col('tpep_pickup_datetime').cast("long")) / 60)

print('Median duration of taxi trip leaving Midtown headed for JFK Airport in 2018 : {:.2f} minutes.'.format(df_jfk_to_midtown.approxQuantile("time_duration", [0.5], 0.1)[0]))

## Geographic information

For this, you will need to find tools to display maps and to build choropeth maps.
We let you look and find relevant tools to do this.

1. Build a heatmap where color is a function of
    1. number of `pickups`
    2. number of `dropoffs`
    3. number of `pickups` with dropoff at some airport (JFK, LaGuardia, Newark)
2. Build a choropeth map where color is a function of
    1. number of pickups in the area
    1. ratio of number of payments by card/number of cash payments for pickups in the area
    2. ratio of total fare/trip duration for dropoff in the area
3. Build an interactive chorophet with a slider allowing the user to select an `hour of day` and where the color is a function of
    1. average number of dropoffs in the area during that hour the day
    2. average ratio of tip over total fare amount for pickups in the area at given hour of the day

In [ ]:
# DISCLAIMER 2

# Nous n'avons fait que les questions de la partie 1 avec le dataset de 2015 et seulement les questions
# des parties 2 et 3 avec le dataset de 2018.

# La raison est que le dataset de 2015 contient seulement les coordonées géographiques et celui de 2018
# seulement les ID des taxis zones.

# Convertir les coordonées en taxis zones est potentiellement faisable, mais requiert beaucoup de temps et de code
# très répétitif.
# Convertir les taxis zones en coordonées est plus facile, mais résulterait en une perte d'information conséquente :
# les coordonées de chaque zone seraient les mêmes, les locations seraient donc tous assemblées par zones.

In [ ]:
# 3.3.1.A - 2015

fig = px.density_mapbox(df_2015.sample(0.02).toPandas(), lat='pickup_latitude', lon='pickup_longitude', radius=1,
                        center=dict(lat=40.72, lon=-73.87), zoom=10,
                        mapbox_style="carto-darkmatter")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# 3.3.1.B - 2015

fig = px.density_mapbox(df_2015.sample(0.02).toPandas(), lat='dropoff_latitude', lon='dropoff_longitude', radius=1,
                        center=dict(lat=40.72, lon=-73.87), zoom=10,
                        mapbox_style="carto-darkmatter")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# 3.3.1.C - 2015

long_min_JFK = -73.82366180419922
long_max_JFK = -73.7431526184082
lat_min_JFK = 40.61799193991041
lat_max_JFK = 40.66944158747386

long_min_LG = -73.89344215393066
long_max_LG = -73.8507843017578
lat_min_LG = 40.764746389047964
lat_max_LG = 40.787365288545566

long_min_NW = -74.19427871704102
long_max_NW = -74.15033340454102
lat_min_NW = 40.66918118282895
lat_max_NW = 40.711483593215306
 
df_FJK = df_2015.where(df_2015.pickup_longitude > long_min_JFK).where(df_2015.pickup_longitude < long_max_JFK)\
                .where(df_2015.pickup_latitude > lat_min_JFK).where(df_2015.pickup_latitude < lat_max_JFK)

df_LG = df_2015.where(df_2015.pickup_longitude > long_min_LG).where(df_2015.pickup_longitude < long_max_LG)\
                .where(df_2015.pickup_latitude > lat_min_LG).where(df_2015.pickup_latitude < lat_max_LG)

df_NW = df_2015.where(df_2015.pickup_longitude > long_min_NW).where(df_2015.pickup_longitude < long_max_NW)\
                .where(df_2015.pickup_latitude > lat_min_NW).where(df_2015.pickup_latitude < lat_max_NW)


df = df_FJK.union(df_LG).union(df_NW)
df = df.select('dropoff_latitude', 'dropoff_longitude')

fig = px.density_mapbox(df.sample(0.3).toPandas(), lat='dropoff_latitude', lon='dropoff_longitude', radius=1,
                        center=dict(lat=40.72, lon=-73.87), zoom=10,
                        mapbox_style="carto-darkmatter")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# 3.3.2.A - 2018

# On charge les données GéoJSON des zones de New York
# Source : https://data.cityofnewyork.us/api/geospatial/d3c5-ddgc?method=export&format=GeoJSON
location = json.load(open('nyc_taxi_zones.geojson', 'r'))

# On compte le nombre de pickup dans chaque zone et on complete avec des 0 pour les zones vides
df = df_2018.groupBy('PULocationID').count()
df_zeros = spark.createDataFrame(
    [
        (103, 0),
        (104, 0),
        (110, 0)
    ],
    StructType(
        [
            StructField("PULocationID", IntegerType(), True),
            StructField("count", LongType(), True),
        ]
    ),
)
df = df.union(df_zeros).orderBy('PULocationID')

fig = px.choropleth_mapbox(df.toPandas(), geojson=location, featureidkey='properties.location_id', locations='PULocationID', color='count',
                           mapbox_style="carto-positron",
                           zoom=10, center = {"lat": 40.72, "lon": -73.87},
                           opacity=0.5)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# 3.3.2.B - 2018

df = df_2018.select('PULocationID', 'payment_type').where(df_2018.payment_type <= 2)
df = df.groupBy('PULocationID', 'payment_type').count()

dff = spark.range(1, 266).withColumnRenamed('id', 'PULocationID')
dfff = spark.range(1,3).withColumnRenamed('id', 'payment_type')
dff = dff.crossJoin(dfff)
dgg = spark.range(0,1).withColumnRenamed('id', 'cc')
dff = dff.crossJoin(dgg)
df = dff.join(df, on=['PULocationID', 'payment_type'], how='left')
df = df.orderBy('PULocationID', 'payment_type').withColumnRenamed('count', 'tot')
df = df.withColumn('total', fn.when(df['tot'].isNull(), df.cc).otherwise(df.tot)).drop('cc', 'tot')
df_1 = df.where(df.payment_type == 1).orderBy('PULocationID').withColumnRenamed('total', 'total_1').drop('payment_type')
df_2 = df.where(df.payment_type == 2).orderBy('PULocationID').withColumnRenamed('total', 'total_2').drop('payment_type')

df = df_1.join(df_2, on=['PULocationID'])
df = df.withColumn('ratio', df.total_1 / (df.total_1 + df.total_2)).orderBy('PULocationID').drop('total_1', 'total_2')

fig = px.choropleth_mapbox(df.toPandas(), geojson=location, featureidkey='properties.location_id', locations='PULocationID', color='ratio',
                           mapbox_style="carto-positron",
                           zoom=10, center = {"lat": 40.72, "lon": -73.87},
                           opacity=0.5)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# 3.3.2.C - 2018

df = df_2018.select('DOLocationID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'fare_amount')
df = df.withColumn('time_duration', (col("tpep_dropoff_datetime").cast("long") - col('tpep_pickup_datetime').cast("long")) / 60)
df = df.groupBy('DOLocationID').sum('fare_amount', 'time_duration')

df = df.withColumn('ratio', df['sum(fare_amount)'] / df['sum(time_duration)'])
df = df.drop('sum(fare_amount)', 'sum(time_duration)')

fig = px.choropleth_mapbox(df.toPandas(), geojson=location, featureidkey='properties.location_id', locations='DOLocationID', color='ratio',
                           mapbox_style="carto-positron",
                           zoom=10, center = {"lat": 40.72, "lon": -73.87},
                           opacity=0.5)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# 3.3.3.A - 2018

df = df_2018.select('DOLocationID', 'tpep_dropoff_datetime')
df = df.withColumn('hour', fn.hour(df.tpep_dropoff_datetime))
df = df.drop('tpep_dropoff_datetime').groupBy('DOLocationID', 'hour').count()
df = df.orderBy('DOLocationID', 'hour')

fig = px.choropleth_mapbox(df.toPandas(), geojson=location, featureidkey='properties.location_id', locations='DOLocationID', color='count',
                           mapbox_style="carto-positron", animation_frame='hour',
                           zoom=10, center = {"lat": 40.72, "lon": -73.87},
                           opacity=0.5)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# 3.3.3.B - 2018

df = df_2018.select('PULocationID', 'tpep_pickup_datetime', 'fare_amount', 'tip_amount')
df = df.withColumn('hour', fn.hour(df.tpep_pickup_datetime)).drop('tpep_pickup_datetime') 
df = df.withColumn('ratio', df.tip_amount / df.fare_amount).drop('fare_amount', 'tip_amount')
df = df.groupBy('PULocationID', 'hour').avg('ratio').withColumnRenamed('avg(ratio)', 'ratio')
df = df.orderBy('PULocationID', 'hour')

fig = px.choropleth_mapbox(df.toPandas(), geojson=location, featureidkey='properties.location_id', locations='PULocationID', color='ratio',
                           mapbox_style="carto-positron", animation_frame='hour',
                           zoom=10, center = {"lat": 40.72, "lon": -73.87},
                           opacity=0.5)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()